<a href="https://colab.research.google.com/github/jbastian9/SeminarioBigData/blob/main/Copia_de_Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=b5b09df303a6dcb4e641ec983efc01364bb82fa5d4698ad88132f66f75e2df51
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


El archivo CSV presenta errores de formato al intentar separar los datos utilizando el delimitador ",". Por esta razón, decidimos utilizar un RDD con un split por coma para cada elemento del CSV.

Cargar el archivo como un Resilient Distributed Dataset (RDD) nos permite manipular el contenido del archivo a un nivel más bajo que utilizando un DataFrame. Esto puede ser útil cuando el archivo tiene problemas de formato, como es nuestro caso actual.

Es importante tener en cuenta que al convertir el archivo CSV en un TXT y usar un split por ",", este método podría no ser útil si la información incluye campos como "observaciones" u otros tipos de contenido que puedan contener comas.

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

# Iniciar una sesión de Spark
spark = SparkSession.builder.appName("LimpiezaDeDatos").getOrCreate()

# Ruta del archivo CSV
ruta_archivo = "/content/drive/MyDrive/Colab Notebooks/data/Copia de housing_fincaraiz.csv"

# Cargar el archivo como un RDD
rdd = spark.sparkContext.textFile(ruta_archivo)

# Separar el encabezado del resto de los datos
encabezado = rdd.first()
datos = rdd.filter(lambda x: x != encabezado)

# Dividir el encabezado y las filas por comas
columnas = encabezado.split(',')

# Asegurarse de que la longitud de los nombres de las columnas coincide con la cantidad de datos en cada fila
def parsear_fila(fila):
    elementos = fila.split(',')
    if len(elementos) == len(columnas):
        return Row(*elementos)
    else:
        return None

# Parsear las filas y filtrar las que no se pueden parsear correctamente
filas_parsedas = datos.map(parsear_fila).filter(lambda x: x is not None)

# Crear el DataFrame a partir del RDD
df = spark.createDataFrame(filas_parsedas, schema=columnas)

# Nuevos nombres de columnas
nuevos_nombres_columnas = [
    "habitaciones", "baños", "parqueaderos", "area_construida", "area_privada", "estrato",
    "estado", "antiguedad", "administracion", "precio_m2", "ascensor", "circuito_cerrado_de_TV",
    "parqueadero_visitantes", "porteria_recepcion", "zonas_verdes", "salon_comunal", "balcon",
    "barra_estilo_americano", "calentador", "chimenea", "citofono", "cocina_integral", "terraza",
    "vigilancia", "parques_cercanos", "estudio", "patio", "deposito_bodega", "nombre", "ubicacion", "precio"
]

# Validación de la cantidad de columnas
if len(columnas) != len(nuevos_nombres_columnas):
    print("The number of columns does not match. Adjust the new_column_names list.")
    print(f"Original Columns ({len(columnas)}): {columnas}")
    print(f"New Column Names ({len(nuevos_nombres_columnas)}): {nuevos_nombres_columnas}")
else:
    # Renombrar las columnas del DataFrame
    df = df.toDF(*nuevos_nombres_columnas)
    # Mostrar todos los datos en forma de tabla
    df.show(truncate=False)


+------------+-----+------------+---------------+------------+-------+-----------+--------------+---------------+--------------+--------+----------------------+----------------------+------------------+------------+-------------+------+----------------------+----------+--------+--------+---------------+-------+----------+----------------+-------+-----+---------------+-----------+--------------------+----------+
|habitaciones|baños|parqueaderos|area_construida|area_privada|estrato|estado     |antiguedad    |administracion |precio_m2     |ascensor|circuito_cerrado_de_TV|parqueadero_visitantes|porteria_recepcion|zonas_verdes|salon_comunal|balcon|barra_estilo_americano|calentador|chimenea|citofono|cocina_integral|terraza|vigilancia|parques_cercanos|estudio|patio|deposito_bodega|nombre     |ubicacion           |precio    |
+------------+-----+------------+---------------+------------+-------+-----------+--------------+---------------+--------------+--------+----------------------+----------

In [8]:
from pyspark.sql.functions import col, regexp_replace, lower
import unicodedata

# Limpiar espacios en blanco al final y convertir a minúsculas
df_cleaned = df.select([
    lower(regexp_replace(col(column), '\s+$', '')).alias(column) for column in df.columns
])

# Aplicar limpieza específica para las columnas de precios y áreas
df_cleaned = df_cleaned.withColumn('precio_m2', regexp_replace('precio_m2', '[^\d]', ''))
df_cleaned = df_cleaned.withColumn('administracion', regexp_replace('administracion', '[^\d]', ''))
df_cleaned = df_cleaned.withColumn('precio', regexp_replace('precio', '[^\d]', ''))

# Convertir las columnas de áreas a números eliminando el indicador de m²
df_cleaned = df_cleaned.withColumn('area_construida', regexp_replace('area_construida', '[^\d]', ''))
df_cleaned = df_cleaned.withColumn('area_privada', regexp_replace('area_privada', '[^\d]', ''))

# Mostrar la tabla limpia
df_cleaned.show(truncate=False)


+------------+-----+------------+---------------+------------+-------+-----------+--------------+--------------+---------+--------+----------------------+----------------------+------------------+------------+-------------+------+----------------------+----------+--------+--------+---------------+-------+----------+----------------+-------+-----+---------------+-----------+--------------------+----------+
|habitaciones|baños|parqueaderos|area_construida|area_privada|estrato|estado     |antiguedad    |administracion|precio_m2|ascensor|circuito_cerrado_de_TV|parqueadero_visitantes|porteria_recepcion|zonas_verdes|salon_comunal|balcon|barra_estilo_americano|calentador|chimenea|citofono|cocina_integral|terraza|vigilancia|parques_cercanos|estudio|patio|deposito_bodega|nombre     |ubicacion           |precio    |
+------------+-----+------------+---------------+------------+-------+-----------+--------------+--------------+---------+--------+----------------------+----------------------+-----